A rough copy of https://towardsdatascience.com/text-classification-with-bert-in-pytorch-887965e5820f

Download dataset from https://www.kaggle.com/c/learn-ai-bbc/data

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch import nn
from transformers import BertTokenizer, BertModel
from helpers import get_device

In [2]:
torch.manual_seed(0)
np.random.seed(0)
device = get_device()

In [3]:
data_dir = Path(".").resolve(strict=True).parent / "data"
train_df = pd.read_csv(data_dir / "BBC News Train.csv")
labels = {"business": 0, "entertainment": 1, "sport": 2, "tech": 3, "politics": 4}
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.labels = [labels[category] for category in data["Category"]]
        self.texts = [
            tokenizer(
                text,
                padding="max_length",
                max_length=512,
                truncation=True,
                return_tensors="pt",
            )
            for text in data["Text"]
        ]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], np.array(self.labels[idx])


class BERTClassifier(nn.Module):
    def __init__(self, dropout_p=0.5):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-cased")
        self.dropout = nn.Dropout(dropout_p)
        self.fc = nn.Linear(768, 5)

    def __call__(self, input_ids, mask):
        _, output = self.bert(input_ids=input_ids, attention_mask=mask, return_dict=False)
        output = self.fc(self.dropout(output)).relu()
        return output

In [5]:
train_df, valid_df, test_df = np.split(
    train_df.sample(frac=1, random_state=1337), [int(0.8*len(train_df)), int(0.9*len(train_df))]
)
train_dataloader = torch.utils.data.DataLoader(Dataset(train_df), batch_size=2, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(Dataset(valid_df), batch_size=2)
criterion = nn.CrossEntropyLoss()
model = BERTClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)

/Users/jacky/repos/fromthetensor/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
for epoch in range(5):
    total_train_loss = 0
    total_train_correct = 0
    for train_input, train_label in tqdm(train_dataloader):
        mask = train_input["attention_mask"].to(device)
        input_id = train_input["input_ids"].squeeze(1).to(device)
        output = model(input_id, mask)
        train_label = train_label.to(device)
        loss = criterion(output, train_label.long())
        total_train_loss += loss.item()
        total_train_correct += (output.argmax(dim=1) == train_label).sum().item()
        model.zero_grad()
        loss.backward()
        optimizer.step()
    total_valid_loss = 0
    total_valid_correct = 0
    with torch.no_grad():
        for valid_input, valid_label in valid_dataloader:
            mask = valid_input["attention_mask"].to(device)
            input_ids = valid_input["input_ids"].squeeze(1).to(device)
            output = model(input_ids, mask)
            valid_label = valid_label.to(device)
            loss = criterion(output, valid_label.long())
            total_valid_loss += loss.item()
            total_valid_correct += (output.argmax(dim=1) == valid_label).sum().item()
    print(
        f"epoch {epoch+1} | "
        f"train loss {total_train_loss / len(train_df):.4f} | "
        f"train accuracy {total_train_correct / len(train_df):.4f} | "
        f"valid loss {total_valid_loss / len(valid_df):.4f} | "
        f"valid accuracy {total_valid_correct / len(valid_df):.4f}"
    )

100%|██████████| 596/596 [03:07<00:00,  3.18it/s]


epoch 1 | train loss 0.7920 | train accuracy 0.2802 | valid loss 0.7140 | valid accuracy 0.5436


100%|██████████| 596/596 [03:10<00:00,  3.12it/s]


epoch 2 | train loss 0.5085 | train accuracy 0.7659 | valid loss 0.3167 | valid accuracy 0.9799


100%|██████████| 596/596 [03:19<00:00,  2.99it/s]


epoch 3 | train loss 0.2334 | train accuracy 0.9664 | valid loss 0.1624 | valid accuracy 0.9933


100%|██████████| 596/596 [03:12<00:00,  3.09it/s]


epoch 4 | train loss 0.1338 | train accuracy 0.9790 | valid loss 0.1124 | valid accuracy 0.9866


100%|██████████| 596/596 [03:12<00:00,  3.10it/s]


epoch 5 | train loss 0.0836 | train accuracy 0.9908 | valid loss 0.0724 | valid accuracy 0.9799


In [7]:
test_dataloader = torch.utils.data.DataLoader(Dataset(test_df), batch_size=2)
total_test_correct = 0
with torch.no_grad():
    for test_input, test_label in test_dataloader:
        mask = test_input["attention_mask"].to(device)
        input_ids = test_input["input_ids"].squeeze(1).to(device)
        output = model(input_ids, mask)
        test_label = test_label.to(device)
        total_test_correct += (output.argmax(dim=1) == test_label).sum().item()
print(f"test accuracy {total_test_correct / len(test_df):.4f}")

test accuracy 0.9530
